# k-Means Initialisierung - Warum Startpunkte wichtig sind

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_19/kmeans_initialization_experiment.ipynb)

In diesem Notebook experimentieren wir mit **schwierigen Clustern** und sehen, warum die **Initialisierung** bei k-Means so wichtig ist.

**Das Problem:** k-Means kann bei ungünstigen Startpunkten in **lokalen Minima** stecken bleiben und schlechte Ergebnisse liefern.

**Heute experimentieren wir:**
1. 🎯 Schwierige Cluster erstellen (realistisch!)
2. 🎲 Verschiedene Initialisierungen ausprobieren
3. 🔄 Bedeutung von `n_init` verstehen
4. 🧪 Selbst experimentieren und Unterschiede sehen

**Interaktiv:** Ihr könnt verschiedene `random_state` Werte ausprobieren und sehen, wie unterschiedlich die Ergebnisse werden!

## 1. Bibliotheken importieren

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Für reproduzierbare Plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("🎯 k-Means Initialisierungs-Experiment geladen!")
print("Heute lernen wir, warum Startpunkte so wichtig sind!")

## 2. Schwierige Cluster erstellen

Wir erstellen **absichtlich schwierige Daten** mit drei überlappenden Clustern. Das ist realistischer als perfekt getrennte Cluster!

In [ ]:
# Bananenförmige Cluster - realistisch und schwierig für k-Means!
np.random.seed(42)

# Cluster 1: Obere Banane (gebogen nach oben)
t1 = np.linspace(0, np.pi, 100)
cluster1_x = 3 * t1 + np.random.normal(0, 0.3, 100)
cluster1_y = 2 * np.sin(t1) + 2 + np.random.normal(0, 0.2, 100)
cluster1 = np.column_stack([cluster1_x, cluster1_y])

# Cluster 2: Untere Banane (gebogen nach unten)  
t2 = np.linspace(0, np.pi, 100)
cluster2_x = 3 * t2 + np.random.normal(0, 0.3, 100)
cluster2_y = -2 * np.sin(t2) - 2 + np.random.normal(0, 0.2, 100)
cluster2 = np.column_stack([cluster2_x, cluster2_y])

# Cluster 3: Seitliche Banane (gebogen nach rechts)
t3 = np.linspace(-np.pi/2, np.pi/2, 80)
cluster3_x = 2 * np.cos(t3) - 2 + np.random.normal(0, 0.2, 80)
cluster3_y = 3 * t3 + np.random.normal(0, 0.3, 80)
cluster3 = np.column_stack([cluster3_x, cluster3_y])

X = np.vstack([cluster1, cluster2, cluster3])
y_true = np.hstack([np.zeros(100), np.ones(100), np.full(80, 2)])

print(f"📊 BANANENFÖRMIGE CLUSTER - schwierig aber lösbar für k-Means:")
print(f"   • {len(X)} Datenpunkte")  
print(f"   • Cluster 1: Obere Banane (100 Punkte)")
print(f"   • Cluster 2: Untere Banane (100 Punkte)")
print(f"   • Cluster 3: Seitliche Banane (80 Punkte)")
print(f"   • Gebogene Cluster sind schwierig - Initialisierung wird kritisch!")

# Echte Cluster visualisieren
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green']
for i in range(3):
    cluster_points = X[y_true == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=50, 
               label=f'Echter Cluster {i+1}')

plt.title('BANANENFÖRMIGE CLUSTER - Eine realistische Herausforderung!', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("👁️ Beobachtung: Bananenförmige Cluster sind realistisch und herausfordernd!")
print("k-Means bevorzugt runde Cluster - gebogene Formen sind schwieriger!")

## 3. Das Problem demonstrieren: Schlechte vs. gute Initialisierung

Zuerst zeigen wir, dass k-Means bei ungünstigen Startpunkten schlechte Ergebnisse liefern kann.

In [ ]:
# Feste schlechte und gute Startzentren für konsistente Demonstration
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Schlechte Initialisierung - ungünstig verteilte Zentren
start_centers_bad = np.array([
    [3.9, -0.8],   # Ungünstige Verteilung der Start-Zentren
    [-2.0, -3.9],  # → führt zu schlechter Clusteraufteilung
    [-1.9, 1.2]    # → k-Means findet lokales Minimum
])
kmeans_bad = KMeans(n_clusters=3, init=start_centers_bad, n_init=1, random_state=0)
labels_bad = kmeans_bad.fit_predict(X)
centers_bad = kmeans_bad.cluster_centers_

# Gute Initialisierung - strategisch platzierte Start-Zentren
start_centers_good = np.array([
    [4.5, 2.0],    # Mitte der oberen Banane  
    [4.5, -2.0],   # Mitte der unteren Banane
    [-1.0, 0.0]    # Mitte der seitlichen Banane
])
kmeans_good = KMeans(n_clusters=3, init=start_centers_good, n_init=1, random_state=0)
labels_good = kmeans_good.fit_predict(X)
centers_good = kmeans_good.cluster_centers_

# Plot 1: Schlechte Initialisierung
# Einfache Lösung: Cluster nach Zentren-Position sortieren für Konsistenz
centers_bad_sorted = np.argsort(centers_bad[:, 0])  # Nach x-Koordinate sortieren
for i, cluster_idx in enumerate(centers_bad_sorted):
    cluster_points = X[labels_bad == cluster_idx]
    ax1.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=30)
ax1.scatter(centers_bad[:, 0], centers_bad[:, 1], 
           c='black', marker='x', s=200, linewidths=3, label='Finale Zentren')
ax1.scatter(start_centers_bad[:, 0], start_centers_bad[:, 1], 
           c='black', marker='s', s=100, alpha=0.8, label='Start-Zentren')
ax1.set_title('Schlechte Initialisierung\n(fällt in lokales Minimum)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Gute Initialisierung
centers_good_sorted = np.argsort(centers_good[:, 0])  # Nach x-Koordinate sortieren
for i, cluster_idx in enumerate(centers_good_sorted):
    cluster_points = X[labels_good == cluster_idx]
    ax2.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=30)
ax2.scatter(centers_good[:, 0], centers_good[:, 1], 
           c='black', marker='x', s=200, linewidths=3, label='Finale Zentren')
ax2.scatter(start_centers_good[:, 0], start_centers_good[:, 1], 
           c='black', marker='s', s=100, alpha=0.8, label='Start-Zentren')
ax2.set_title('Gute Initialisierung\n(findet globales Minimum)', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Qualität vergleichen
ari_bad = adjusted_rand_score(y_true, labels_bad)
ari_good = adjusted_rand_score(y_true, labels_good)
sil_bad = silhouette_score(X, labels_bad)
sil_good = silhouette_score(X, labels_good)

print("📊 Qualitätsvergleich:")
print(f"Schlechte Init: ARI = {ari_bad:.3f}, Silhouette = {sil_bad:.3f}")
print(f"Gute Init:      ARI = {ari_good:.3f}, Silhouette = {sil_good:.3f}")
print(f"Verbesserung:   ARI = +{ari_good-ari_bad:.3f}, Silhouette = +{sil_good-sil_bad:.3f}")
print()
print("💡 Erkenntnisse:")
print("   • Bananenförmige Cluster zeigen warum Initialisierung wichtig ist!")
print("   • Schlechte Initialisierung: Alle Zentren in einer Banane gefangen")
print("   • k-Means teilt dann andere Bananen willkürlich auf")  
print("   • Gute Initialisierung: Ein Zentrum pro Banane → bessere Lösung")
print("   • Gebogene Strukturen bleiben schwierig, aber lösbar")
print("   • Das zeigt: Initialisierung kann entscheidend sein!")
print("   • Realistisches Beispiel: Kundensegmente, Bildregionen, etc.")

## 4. Die Rettung: n_init Parameter

**Das Problem:** Eine einzelne zufällige Initialisierung kann Pech haben.  
**Die Lösung:** `n_init` führt k-Means mehrmals mit verschiedenen Startpunkten aus und wählt das beste Ergebnis!

In [ ]:
# n_init Experiment
print("🔄 n_init Experiment - Mehrere Versuche = bessere Ergebnisse")
print("="*60)

n_init_values = [1, 5, 10, 20, 50]
results = []

for n_init in n_init_values:
    # k-Means mit verschiedenen n_init Werten
    kmeans = KMeans(n_clusters=3, n_init=n_init, random_state=42)
    labels = kmeans.fit_predict(X)
    
    # Qualität messen
    ari = adjusted_rand_score(y_true, labels)
    silhouette = silhouette_score(X, labels)
    inertia = kmeans.inertia_
    
    results.append({
        'n_init': n_init,
        'ari': ari,
        'silhouette': silhouette,
        'inertia': inertia,
        'labels': labels,
        'centers': kmeans.cluster_centers_
    })
    
    print(f"n_init={n_init:2d}: ARI={ari:.3f}, Silhouette={silhouette:.3f}, Inertia={inertia:.1f}")

print("="*60)
print("👁️ Beobachtung: Mit höherem n_init werden die Ergebnisse stabiler und besser!")

# Bestes Ergebnis visualisieren
best_result = max(results, key=lambda x: x['ari'])
print(f"\n🏆 Bestes Ergebnis bei n_init={best_result['n_init']}")

plt.figure(figsize=(12, 4))

# Plot 1: n_init=1 (schlechtes Ergebnis)
plt.subplot(1, 3, 1)
worst_result = min(results, key=lambda x: x['ari'])
# Einfache Sortierung für Konsistenz
centers_sorted = np.argsort(worst_result['centers'][:, 0])
for i, cluster_idx in enumerate(centers_sorted):
    cluster_points = X[worst_result['labels'] == cluster_idx]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=30)
plt.scatter(worst_result['centers'][:, 0], worst_result['centers'][:, 1], 
           c='black', marker='x', s=150, linewidths=2)
plt.title(f'n_init={worst_result["n_init"]}\nARI={worst_result["ari"]:.3f}', fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot 2: n_init=10 (mittleres Ergebnis)
plt.subplot(1, 3, 2)
mid_result = [r for r in results if r['n_init'] == 10][0]
centers_sorted = np.argsort(mid_result['centers'][:, 0])
for i, cluster_idx in enumerate(centers_sorted):
    cluster_points = X[mid_result['labels'] == cluster_idx]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=30)
plt.scatter(mid_result['centers'][:, 0], mid_result['centers'][:, 1], 
           c='black', marker='x', s=150, linewidths=2)
plt.title(f'n_init={mid_result["n_init"]}\nARI={mid_result["ari"]:.3f}', fontweight='bold')
plt.grid(True, alpha=0.3)

# Plot 3: Bestes Ergebnis
plt.subplot(1, 3, 3) 
centers_sorted = np.argsort(best_result['centers'][:, 0])
for i, cluster_idx in enumerate(centers_sorted):
    cluster_points = X[best_result['labels'] == cluster_idx]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
               c=colors[i], alpha=0.6, s=30)
plt.scatter(best_result['centers'][:, 0], best_result['centers'][:, 1], 
           c='black', marker='x', s=150, linewidths=2)
plt.title(f'n_init={best_result["n_init"]}\nARI={best_result["ari"]:.3f}', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 🧪 EXPERIMENTIER-SECTION: Jetzt seid ihr dran!

**Aufgabe:** Probiert verschiedene `random_state` Werte aus und seht, wie unterschiedlich die Ergebnisse werden können!

**Anleitung:**
1. Ändert die `random_state` Werte in der Liste unten
2. Führt die Zelle aus und schaut euch die Ergebnisse an
3. Probiert verschiedene Werte aus: 1, 7, 13, 42, 99, 123, 999...
4. Beobachtet: Welche Werte geben gute/schlechte Ergebnisse?

In [ ]:
# 🧪 EXPERIMENTIER-BEREICH - HIER KÖNNT IHR EXPERIMENTIEREN!

# Diese Werte könnt ihr ändern:
experiment_random_states = [1, 7, 42, 99, 123]  # ← HIER EXPERIMENTIEREN!
experiment_n_init = 1  # ← Erstmal auf 1 lassen, dann auf 10 ändern

print("🧪 EURE EXPERIMENTE:")
print(f"Testet random_state Werte: {experiment_random_states}")
print(f"Mit n_init = {experiment_n_init}")
print("="*60)

experiment_results = []

for rs in experiment_random_states:
    # k-Means mit eurem random_state
    kmeans = KMeans(n_clusters=3, random_state=rs, n_init=experiment_n_init)
    labels = kmeans.fit_predict(X)
    
    # Qualität messen
    ari = adjusted_rand_score(y_true, labels)
    silhouette = silhouette_score(X, labels)
    
    experiment_results.append({
        'random_state': rs,
        'ari': ari,
        'silhouette': silhouette,
        'labels': labels,
        'centers': kmeans.cluster_centers_
    })
    
    # Qualitätsbewertung
    if ari > 0.8:
        quality = "Exzellent! 🌟"
    elif ari > 0.6:
        quality = "Gut! ✅"
    elif ari > 0.4:
        quality = "OK 🔶"
    else:
        quality = "Schlecht ❌"
    
    print(f"random_state={rs:3d}: ARI={ari:.3f}, Silhouette={silhouette:.3f} → {quality}")

print("="*60)

# Beste und schlechteste Ergebnisse zeigen
best_exp = max(experiment_results, key=lambda x: x['ari'])
worst_exp = min(experiment_results, key=lambda x: x['ari'])

print(f"🏆 Bestes Ergebnis: random_state={best_exp['random_state']} (ARI={best_exp['ari']:.3f})")
print(f"💀 Schlechtestes Ergebnis: random_state={worst_exp['random_state']} (ARI={worst_exp['ari']:.3f})")
print(f"📊 Unterschied: {best_exp['ari'] - worst_exp['ari']:.3f} ARI Punkte!")

# Visualisierung der Experimente
n_experiments = len(experiment_results)
fig, axes = plt.subplots(1, min(n_experiments, 5), figsize=(4*min(n_experiments, 5), 4))

if n_experiments == 1:
    axes = [axes]

for i, result in enumerate(experiment_results):
    if i >= 5:  # Maximal 5 Plots
        break
        
    ax = axes[i] if len(axes) > 1 else axes[0]
    
    # Cluster zeichnen - sortiert für Konsistenz
    centers_sorted = np.argsort(result['centers'][:, 0])
    for color_idx, cluster_idx in enumerate(centers_sorted):
        cluster_points = X[result['labels'] == cluster_idx]
        ax.scatter(cluster_points[:, 0], cluster_points[:, 1], 
                  c=colors[color_idx], alpha=0.6, s=30)
    
    # Zentren zeichnen
    ax.scatter(result['centers'][:, 0], result['centers'][:, 1], 
              c='black', marker='x', s=150, linewidths=2)
    
    # Titel mit Bewertung
    quality_emoji = "🌟" if result['ari'] > 0.8 else "✅" if result['ari'] > 0.6 else "🔶" if result['ari'] > 0.4 else "❌"
    ax.set_title(f'random_state={result["random_state"]}\nARI={result["ari"]:.3f} {quality_emoji}', 
                fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 EXPERIMENTIER-TIPPS:")
print("• Ändert die random_state Werte und führt die Zelle erneut aus!")
print("• Probiert: 1, 7, 13, 42, 77, 99, 123, 456, 789, 999")
print("• Danach: Setzt experiment_n_init = 10 und schaut den Unterschied!")
print("• Beobachtet: Welche Werte sind stabil gut/schlecht?")

## 6. 🎯 Erweiterte Experimente: Verschiedene Initialisierungsmethoden

k-Means bietet verschiedene Initialisierungsmethoden. Probiert sie aus!

In [ ]:
# Verschiedene Initialisierungsmethoden testen
print("🎯 Verschiedene Initialisierungsmethoden:")
print("="*50)

init_methods = ['k-means++', 'random']  # Die zwei wichtigsten Methoden
method_results = {}

for method in init_methods:
    print(f"\n📊 Methode: {method}")
    print("-" * 30)
    
    method_aris = []
    method_silhouettes = []
    
    # 10 Versuche mit verschiedenen random_states
    for rs in range(10):
        kmeans = KMeans(n_clusters=3, init=method, random_state=rs, n_init=1)
        labels = kmeans.fit_predict(X)
        
        ari = adjusted_rand_score(y_true, labels)
        silhouette = silhouette_score(X, labels)
        
        method_aris.append(ari)
        method_silhouettes.append(silhouette)
        
        print(f"  Versuch {rs+1:2d}: ARI={ari:.3f}, Silhouette={silhouette:.3f}")
    
    # Statistiken berechnen
    avg_ari = np.mean(method_aris)
    std_ari = np.std(method_aris)
    min_ari = np.min(method_aris)
    max_ari = np.max(method_aris)
    
    method_results[method] = {
        'aris': method_aris,
        'silhouettes': method_silhouettes,
        'avg_ari': avg_ari,
        'std_ari': std_ari,
        'min_ari': min_ari,
        'max_ari': max_ari
    }
    
    print(f"  📈 Durchschnitt: {avg_ari:.3f} ± {std_ari:.3f}")
    print(f"  📊 Bereich: {min_ari:.3f} - {max_ari:.3f}")

print("\n" + "="*50)
print("🏆 VERGLEICH DER METHODEN:")

for method in init_methods:
    results = method_results[method]
    stability = "stabil" if results['std_ari'] < 0.1 else "instabil"
    quality = "gut" if results['avg_ari'] > 0.6 else "mäßig"
    
    print(f"{method:12}: Durchschnitt={results['avg_ari']:.3f}, "
          f"Streuung={results['std_ari']:.3f} → {quality}, {stability}")

# Visualisierung der Verteilungen
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for i, method in enumerate(init_methods):
    aris = method_results[method]['aris']
    plt.hist(aris, alpha=0.7, label=f'{method}', bins=8)
plt.xlabel('ARI Score')
plt.ylabel('Häufigkeit')
plt.title('Verteilung der ARI Scores\n(10 Versuche pro Methode)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
methods = list(method_results.keys())
avgs = [method_results[m]['avg_ari'] for m in methods]
stds = [method_results[m]['std_ari'] for m in methods]

plt.bar(methods, avgs, yerr=stds, capsize=5, alpha=0.7, color=['blue', 'orange'])
plt.ylabel('Durchschnittlicher ARI Score')
plt.title('Methodenvergleich\n(Durchschnitt ± Standardabweichung)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 ERKENNTNISSE:")
print("• k-means++: Intelligente Initialisierung, meist bessere und stabilere Ergebnisse")
print("• random: Zufällige Initialisierung, mehr Variabilität in der Qualität")
print("• Deshalb ist k-means++ der Standard in sklearn!")

## 7. 📚 Zusammenfassung: Was haben wir gelernt?

In [ ]:
print("🎓 WICHTIGE ERKENNTNISSE:")
print("="*50)
print()
print("1️⃣ STARTPUNKTE SIND KRITISCH:")
print("   • k-Means ist sensitiv gegenüber der Initialisierung")
print("   • Schlechte Startpunkte → lokale Minima → schlechte Ergebnisse")
print("   • Bei überlappenden Clustern besonders problematisch")
print()
print("2️⃣ n_init IST EUER FREUND:")
print(f"   • n_init=1: Riskant, kann Pech haben")
print(f"   • n_init=10 (Standard): Meist ausreichend")
print(f"   • n_init=50: Sehr sicher, aber langsamer")
print("   • k-Means wählt automatisch das beste von n_init Versuchen")
print()
print("3️⃣ INITIALISIERUNGS-METHODEN:")
print("   • 'k-means++': Intelligent, meist bessere Ergebnisse (Standard)")
print("   • 'random': Zufällig, mehr Variabilität")
print("   • k-means++ ist fast immer die bessere Wahl")
print()
print("4️⃣ RANDOM_STATE EXPERIMENTE:")
print("   • Verschiedene random_states → sehr verschiedene Ergebnisse")
print("   • Manche Werte sind 'glücklich', andere 'unglücklich'")
print("   • Deshalb nie nur einen einzigen Versuch machen!")
print()
print("5️⃣ PRAKTISCHE TIPPS:")
print("   • Immer n_init >= 10 verwenden")
print("   • Bei wichtigen Anwendungen: n_init=20 oder höher")
print("   • init='k-means++' beibehalten (ist Standard)")
print("   • Bei wiederholbaren Ergebnissen: random_state setzen")
print("   • Bei schlechten Ergebnissen: mehr n_init versuchen")
print()
print("🏆 IHR HABT ERFOLGREICH GELERNT:")
print("   ✓ Warum k-Means manchmal schlecht ist")
print("   ✓ Wie man es besser macht")
print("   ✓ Warum Sklearn-Defaults meist gut sind")
print("   ✓ Wie man mit Experimenten besser versteht")
print()
print("🚀 Jetzt könnt ihr k-Means RICHTIG einsetzen!")